In [1]:
import torch
import torch.nn as nn
import numpy as np
import random
import gym
import matplotlib.pyplot as plt
import seaborn as sns
import math
from gym import spaces, logger
from gym.utils import seeding
from torch.utils.tensorboard import SummaryWriter
import datetime
import time

In [2]:
"""
Classic cart-pole system implemented by Rich Sutton et al.
Copied from http://incompleteideas.net/sutton/book/code/pole.c
permalink: https://perma.cc/C9ZM-652R

Continuous version by Ian Danforth
"""

class ContinuousCartPoleEnv(gym.Env):
    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second': 50
    }

    def __init__(self):
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole = 0.1
        self.total_mass = (self.masspole + self.masscart)
        self.length = 0.5  # actually half the pole's length
        self.polemass_length = (self.masspole * self.length)
        self.force_mag = 30.0
        self.tau = 0.02  # seconds between state updates
        self.min_action = -1.0
        self.max_action = 1.0

        # Angle at which to fail the episode
        self.theta_threshold_radians = 12 * 2 * math.pi / 360
        self.x_threshold = 2.4

        # Angle limit set to 2 * theta_threshold_radians so failing observation
        # is still within bounds
        high = np.array([
            self.x_threshold * 2,
            np.finfo(np.float32).max,
            self.theta_threshold_radians * 2,
            np.finfo(np.float32).max])

        self.action_space = spaces.Box(
            low=self.min_action,
            high=self.max_action,
            shape=(1,)
        )
        self.observation_space = spaces.Box(-high, high)

        self.seed()
        self.viewer = None
        self.state = None

        self.steps_beyond_done = None

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def stepPhysics(self, force):
        x, x_dot, theta, theta_dot = self.state
        costheta = math.cos(theta)
        sintheta = math.sin(theta)
        temp = (force + self.polemass_length * theta_dot * theta_dot * sintheta) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / \
            (self.length * (4.0/3.0 - self.masspole * costheta * costheta / self.total_mass))
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass
        x = x + self.tau * x_dot
        x_dot = x_dot + self.tau * xacc
        theta = theta + self.tau * theta_dot
        theta_dot = theta_dot + self.tau * thetaacc
        return (x, x_dot, theta, theta_dot)

    def step(self, action):
        assert self.action_space.contains(action), \
            "%r (%s) invalid" % (action, type(action))
        # Cast action to float to strip np trappings
        force = self.force_mag * float(action)
        self.state = self.stepPhysics(force)
        self.timer += 1
        x, x_dot, theta, theta_dot = self.state
        done = x < -self.x_threshold \
            or x > self.x_threshold \
            or theta < -self.theta_threshold_radians \
            or theta > self.theta_threshold_radians \
            or self.timer >= 200
        done = bool(done)

        if not done:
            reward = 1.0
        elif self.steps_beyond_done is None:
            # Pole just fell!
            self.steps_beyond_done = 0
            reward = 1.0
        else:
            if self.steps_beyond_done == 0:
                logger.warn("""
You are calling 'step()' even though this environment has already returned
done = True. You should always call 'reset()' once you receive 'done = True'
Any further steps are undefined behavior.
                """)
            self.steps_beyond_done += 1
            reward = 0.0

        return np.array(self.state), reward, done, {}

    def reset(self):
        self.timer = 0
        self.state = self.np_random.uniform(low=-0.05, high=0.05, size=(4,))
        self.steps_beyond_done = None
        return np.array(self.state)

    def render(self, mode='human'):
        screen_width = 600
        screen_height = 400

        world_width = self.x_threshold * 2
        scale = screen_width /world_width
        carty = 100  # TOP OF CART
        polewidth = 10.0
        polelen = scale * 1.0
        cartwidth = 50.0
        cartheight = 30.0

        if self.viewer is None:
            from gym.envs.classic_control import rendering
            self.viewer = rendering.Viewer(screen_width, screen_height)
            l, r, t, b = -cartwidth / 2, cartwidth / 2, cartheight / 2, -cartheight / 2
            axleoffset = cartheight / 4.0
            cart = rendering.FilledPolygon([(l, b), (l, t), (r, t), (r, b)])
            self.carttrans = rendering.Transform()
            cart.add_attr(self.carttrans)
            self.viewer.add_geom(cart)
            l, r, t, b = -polewidth / 2, polewidth / 2, polelen-polewidth / 2, -polewidth / 2
            pole = rendering.FilledPolygon([(l, b), (l, t), (r, t), (r, b)])
            pole.set_color(.8, .6, .4)
            self.poletrans = rendering.Transform(translation=(0, axleoffset))
            pole.add_attr(self.poletrans)
            pole.add_attr(self.carttrans)
            self.viewer.add_geom(pole)
            self.axle = rendering.make_circle(polewidth / 2)
            self.axle.add_attr(self.poletrans)
            self.axle.add_attr(self.carttrans)
            self.axle.set_color(.5, .5, .8)
            self.viewer.add_geom(self.axle)
            self.track = rendering.Line((0, carty), (screen_width, carty))
            self.track.set_color(0, 0, 0)
            self.viewer.add_geom(self.track)

        if self.state is None:
            return None

        x = self.state
        cartx = x[0] * scale + screen_width / 2.0  # MIDDLE OF CART
        self.carttrans.set_translation(cartx, carty)
        self.poletrans.set_rotation(-x[2])

        return self.viewer.render(return_rgb_array=(mode == 'rgb_array'))

    def close(self):
        if self.viewer:
            self.viewer.close()

In [142]:
class Policy(nn.Module):
    def __init__(self, num_inputs, num_actions):
        super(Policy, self).__init__()

        # Shared layers
        self.layers = nn.Sequential(
            nn.Linear(num_inputs, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU()
        )
        
        # Individual layers
        self.output_mean = nn.Linear(32, num_actions)
        self.output_std = nn.Linear(32, num_actions)
    
        self.num_inputs = num_inputs
        self.num_actions = num_actions

    def forward(self, x, network_type):
        if network_type == 'mean':
            return torch.tanh(self.output_mean(self.layers(torch.from_numpy(np.asarray(x)).float())))
        else:
            return 0.01 + 100*torch.sigmoid(self.output_std(self.layers(0*torch.from_numpy(np.asarray(x)).float())))

    def act(self, state):
        action = np.random.multivariate_normal(self.forward(state,'mean').detach().numpy(), np.diag(self.forward(state,'std').detach().numpy()))
        return np.clip(action,-1,1)

In [132]:
class ValueFunction(nn.Module):
    def __init__(self, num_inputs):
        super(ValueFunction, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(num_inputs, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

        self.num_inputs = num_inputs

    def forward(self, x):
        return self.layers(torch.from_numpy(np.asarray(x)).float())

In [133]:
NUM_ALGO_TRIALS = 4
NUM_ITERATIONS = 600
ROLLOUT_LENGTH = 500
NUM_TESTS = 1
LEARNING_RATE = .00001
LEARNING_RATE_V = .0001

STATE_DIM = 4
ACTION_DIM = 1

In [134]:
random.seed(17)
np.random.seed(17)
torch.manual_seed(17)

In [138]:
env = ContinuousCartPoleEnv()

# REINFORCE with (adaptive) Value Function Baseline and Aggregated Gradients

In [137]:
LEARNING_RATE = .0000005
LEARNING_RATE_V = .00005

# Run the algorithm multiple times to see performance across random seeds
performance = np.zeros((NUM_ALGO_TRIALS,NUM_ITERATIONS))

for algo_trial in range(NUM_ALGO_TRIALS):
    # Initialise policy as a feedforward NN
    pi = Policy(STATE_DIM, ACTION_DIM)
    v = ValueFunction(STATE_DIM)

    # 1 iteration == deploying a fixed policy in the environment for ROLLOUT_LENGTH timesteps and then 
    # using the collected data from that rollout to perform ROLLOUT_LENGTH policy gradient updates to 
    # find a new policy for the next iteration
    for iteration in range(NUM_ITERATIONS):
        # Always reset the state and history for each new policy
        state = env.reset()
        history = []

        # Collect a rollout using the current policy
        for timestep in range(ROLLOUT_LENGTH): 
            action = pi.act(np.asarray(state).flatten())
            next_state, reward, done, info = env.step(action)
            history.append([state, action, reward, next_state, done])
            state = next_state
            if done:
                state = env.reset()
                
        # Sweep backwards through the collected data and use sample_return to accumulate the rewards 
        # to form the Monte Carlo estimates of the returns
        sample_return = 0
        
        # Reset the gradient aggregators
        param_grads = []
        torch.log(pi(state,'mean') + pi(state,'std')).backward() # need to backprop something before zero_grad()
        pi.zero_grad()
        with torch.no_grad():
            for index, param in enumerate(pi.parameters()):
                param_grads.append(param.grad)
        
        # Sweep back through collected trajectory and aggregate gradients
        for entry in reversed(history):
            state, action, reward, next_state, done = entry
            sample_return += reward

            # Update the Value function baseline
            v_loss = (v(state) - sample_return)**2
            v.zero_grad()
            v_loss.backward()
            
            with torch.no_grad():            
                for index, param in enumerate(v.parameters()):
                    param -= LEARNING_RATE_V * param.grad                                                    

            # Clear the gradients in PyTorch computational graph
            pi.zero_grad()

            # Calculate the gradient of the log-density under the current policy at state, action
            p1 = (1/ torch.sqrt((torch.det(torch.diag(pi(state,'std'))))))
            M = torch.inverse(torch.diag(pi(state,'std')))
            p2 = (torch.from_numpy(action).float() - pi(state,'mean'))
            p3 = p2.T @ M @ p2
            l = torch.log(p1) + (-0.5*p3)
            l.backward()
            
            with torch.no_grad():
                for index, param in enumerate(pi.parameters()):
                    # Aggregate gradients
                    param_grads[index] += LEARNING_RATE * param.grad * (reward + v(next_state) - v(state))
            
            # Clear sample return if done
            if done:
                sample_return = 0

        # After aggregating all the gradients through the log, take a policy gradient step
        with torch.no_grad():
            for index, param in enumerate(pi.parameters()):
                param += param_grads[index]                

        # Reset the state so we can perform an independent evaluation rollout of the updated policy
        state = env.reset()
        evaluation_return = 0
        evaluation_returns = []

        for test_index in range(NUM_TESTS):
            for timestep in range(ROLLOUT_LENGTH):
                action = pi.act(np.asarray(state).flatten())            
                next_state, reward, done, info = env.step(action)
                state = next_state
                evaluation_return += reward
                if done:
                    evaluation_returns.append(evaluation_return)
                    evaluation_return = 0
                    state = env.reset()
                    break
        performance[algo_trial][iteration] = np.mean(evaluation_returns)
        print('REINFORCE Iteration: ', iteration,'\t', 'Evaluation: ', "{:.2f}".format(performance[algo_trial][iteration]) , '\tNum Evals: ', len(evaluation_returns))


REINFORCE Iteration:  0 	 Evaluation:  10.00 	Num Evals:  1
REINFORCE Iteration:  1 	 Evaluation:  7.00 	Num Evals:  1
REINFORCE Iteration:  2 	 Evaluation:  7.00 	Num Evals:  1
REINFORCE Iteration:  3 	 Evaluation:  8.00 	Num Evals:  1
REINFORCE Iteration:  4 	 Evaluation:  7.00 	Num Evals:  1
REINFORCE Iteration:  5 	 Evaluation:  5.00 	Num Evals:  1
REINFORCE Iteration:  6 	 Evaluation:  7.00 	Num Evals:  1
REINFORCE Iteration:  7 	 Evaluation:  6.00 	Num Evals:  1
REINFORCE Iteration:  8 	 Evaluation:  6.00 	Num Evals:  1
REINFORCE Iteration:  9 	 Evaluation:  6.00 	Num Evals:  1
REINFORCE Iteration:  10 	 Evaluation:  6.00 	Num Evals:  1
REINFORCE Iteration:  11 	 Evaluation:  6.00 	Num Evals:  1
REINFORCE Iteration:  12 	 Evaluation:  6.00 	Num Evals:  1
REINFORCE Iteration:  13 	 Evaluation:  6.00 	Num Evals:  1
REINFORCE Iteration:  14 	 Evaluation:  6.00 	Num Evals:  1
REINFORCE Iteration:  15 	 Evaluation:  6.00 	Num Evals:  1
REINFORCE Iteration:  16 	 Evaluation:  6.00 	Num

KeyboardInterrupt: 

In [96]:
pi(state,'mean')

tensor([-0.4442], grad_fn=<AddBackward0>)

In [ ]:
LEARNING_RATE = .00001
LEARNING_RATE_V = .0005

# Run the algorithm multiple times to see performance across random seeds
performance = np.zeros((NUM_ALGO_TRIALS,NUM_ITERATIONS))

for algo_trial in range(NUM_ALGO_TRIALS):
    # Initialise policy as a feedforward NN
    pi = Policy(STATE_DIM, ACTION_DIM)
    v = ValueFunction(STATE_DIM)
    now = datetime.datetime.now()
    writer = SummaryWriter("runs/scalar-{}".format(now.strftime("%Y%m%d-%H%M%S")))

    # 1 iteration == deploying a fixed policy in the environment for ROLLOUT_LENGTH timesteps and then 
    # using the collected data from that rollout to perform ROLLOUT_LENGTH policy gradient updates to 
    # find a new policy for the next iteration
    for iteration in range(NUM_ITERATIONS):
        # Always reset the state and history for each new policy
        state = env.reset()
        history = []

        for timestep in range(ROLLOUT_LENGTH): 
            action = pi.act(np.asarray(state).flatten())
            next_state, reward, done, info = env.step(action)
            reward = reward + 0.5*abs(action.item())
            history.append([state, action, reward, next_state, done])
            state = next_state
            if done:
                state = env.reset()

        # Sweep backwards through the collected data and use sample_return to accumulate the rewards 
        # to form the Monte Carlo estimates of the returns
        sample_return = 0
        likelihoods = []
        for entry in reversed(history):
            state, action, reward, next_state, done = entry
            sample_return += reward

            # Update the Value function baseline
            v_loss = (v(state) - sample_return)**2
            v.zero_grad()
            v_loss.backward()
            
            with torch.no_grad():            
                for index, param in enumerate(v.parameters()):
                    param -= LEARNING_RATE_V * param.grad                                                    

            # Initialise the gradients stored in PyTorch auto-grad computation graph to 0
            pi.zero_grad()

            # Calculate the gradient of the log-density under the current policy at state, action
            p1 = (1/ torch.sqrt((torch.det(torch.diag(pi(state,'std'))))))
            M = torch.inverse(torch.diag(pi(state,'std')))
            p2 = (torch.from_numpy(action).float() - pi(state,'mean'))
            p3 = p2.T @ M @ p2
            l = torch.log(p1) + (-0.5*p3)
            if l.isnan().any():
                raise Exception('About to Blow!')
            l.backward()
            
            with torch.no_grad():
                likelihoods.append(l)
                grads = []
                for index, param in enumerate(pi.parameters()):
                    # Perform a gradient ascent step on the policy parameters
                    param += LEARNING_RATE * param.grad * (sample_return - v(state))
                    grads.extend(param.grad.flatten().tolist())
                writer.add_histogram('policy parameters', torch.nn.utils.parameters_to_vector(pi.parameters()), iteration) 
                writer.add_histogram('policy gradient', np.asarray(grads), iteration)
                writer.add_histogram('log-likelihoods', np.asarray(likelihoods), iteration)                
                writer.flush()
                    
            if done:
                sample_return = 0

                
                
    param_grads = []
    torch.log(pi(state)[action]).backward()
    with torch.no_grad():
        for index, param in enumerate(pi.parameters()):
            param_grads.append(param.grad * 0)

#     for entry in reversed(history):
#         state, action, reward, next_state, done = entry
#         sample_return += reward

#         pi.zero_grad()
#         torch.log(pi(state)[action]).backward()
        
#         with torch.no_grad():
#             for index, param in enumerate(pi.parameters()):
                param_grads[index] += LEARNING_RATE * param.grad * sample_return

#         if done:
#             sample_return = 0
    
    with torch.no_grad():
        for index, param in enumerate(pi.parameters()):
            param += param_grads[index]                
                
                
                
                
                
                
                
                
                
                
                
                
                
        # Reset the state so we can perform an independent evaluation rollout of the updated policy
        state = env.reset()
        evaluation_return = 0
        evaluation_returns = []

        for test_index in range(NUM_TESTS):
            for timestep in range(ROLLOUT_LENGTH):
                action = pi.act(np.asarray(state).flatten())            
                next_state, reward, done, info = env.step(action)
                state = next_state
                evaluation_return += reward
                if done:
                    evaluation_returns.append(evaluation_return)
                    evaluation_return = 0
                    state = env.reset()
                    break
        performance[algo_trial][iteration] = np.mean(evaluation_returns)
        print('REINFORCE Iteration: ', iteration,'\t', 'Evaluation: ', "{:.2f}".format(performance[algo_trial][iteration]) , '\tNum Evals: ', len(evaluation_returns))
    writer.close()        

In [ ]:
l

In [ ]:
vals = []
with torch.no_grad():
    for i in range(1000):
        state = env.reset()
        vals.append(v(state))

In [ ]:
pi(state,'mean')